In [19]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [3]:
file_path = "London_street_trees_gla.csv"
df = pd.read_csv(file_path, low_memory=False)

In [4]:
print(df.head())

    gla_id   borough                            species_name common_name  \
0  glaid_0  Barking   Acer pseudoplatanus 'brilliantissimum'         NaN   
1  glaid_1  Barking                 Taxus baccata fastigiata         NaN   
2  glaid_2  Barking                 Taxus baccata fastigiata         NaN   
3  glaid_3  Barking                 Taxus baccata fastigiata         NaN   
4  glaid_4  Barking                     Robinia pseudoacacia         NaN   

   display_name  load_date   easting  northing  longitude   latitude  
0         Maple   20180214  548320.0  189593.0   0.139454  51.585695  
1         Other   20180214  548297.0  189590.0   0.139123  51.585670  
2         Other   20180214  548289.0  189592.0   0.139004  51.585689  
3         Other   20180214  548298.0  189597.0   0.139134  51.585732  
4  Black Locust   20180214  548308.0  189607.0   0.139288  51.585819  


In [5]:
#Find unique latitude and longitude pairs
unique_coords = df[['latitude', 'longitude']].drop_duplicates()

In [6]:
# Count the unique pairs
unique_count = unique_coords.shape[0]

print(f"Number of unique latitude and longitude pairs: {unique_count}")

Number of unique latitude and longitude pairs: 684159


In [7]:
df.shape[0]

726934

In [10]:
# Count the unique display names
unique_displays = df[['display_name']].drop_duplicates()

unique_displays.shape[0]

24

In [16]:
#group by borough and species, and count the number of trees
tree_boro_spec = df.groupby(['borough', 'species_name']).agg(
    tree_count=('gla_id', 'count'),
    display_name=('display_name', lambda x: ', '.join(x.unique()))  # Concatenate unique display names
).reset_index()

print(tree_boro_spec)

          borough                      species_name  tree_count display_name
0        Barking                     Acer campestre         181        Maple
1        Barking   Acer campestre "queen elizabeth"           1        Maple
2        Barking                  Acer cappadocicum           5        Maple
3        Barking                   Acer cappadocium           1        Maple
4        Barking                       Acer davidii           1        Maple
...           ...                               ...         ...          ...
6255  Westminster                     Ulmus species         110          Elm
6256  Westminster    Viburnum opulus 'xanthocarpum'           2        Other
6257  Westminster                    Viburnum tinus           1        Other
6258  Westminster              Zelkova carpinifolia           2          Elm
6259  Westminster                   Zelkova serrata           5          Elm

[6260 rows x 4 columns]


In [12]:
# Calculate average latitude and longitude for each borough
boro_coords = df.groupby('borough').agg(
    avg_latitude=('latitude', 'mean'),
    avg_longitude=('longitude', 'mean')
).reset_index()

In [17]:
type(tree_boro_spec)

pandas.core.frame.DataFrame

In [18]:
# Merge borough coordinates with tree counts
tree_counts_with_coords = pd.merge(
    tree_boro_spec,
    boro_coords,
    on='borough'
)

print(tree_counts_with_coords)

          borough                      species_name  tree_count display_name  \
0        Barking                     Acer campestre         181        Maple   
1        Barking   Acer campestre "queen elizabeth"           1        Maple   
2        Barking                  Acer cappadocicum           5        Maple   
3        Barking                   Acer cappadocium           1        Maple   
4        Barking                       Acer davidii           1        Maple   
...           ...                               ...         ...          ...   
6255  Westminster                     Ulmus species         110          Elm   
6256  Westminster    Viburnum opulus 'xanthocarpum'           2        Other   
6257  Westminster                    Viburnum tinus           1        Other   
6258  Westminster              Zelkova carpinifolia           2          Elm   
6259  Westminster                   Zelkova serrata           5          Elm   

      avg_latitude  avg_longitude  
0  

In [20]:
# Convert latitude and longitude to geometry points
tree_counts_with_coords["geometry"] = tree_counts_with_coords.apply(lambda row: Point(row["avg_longitude"], row["avg_latitude"]), axis=1)

In [22]:
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(tree_counts_with_coords, geometry="geometry", crs="EPSG:4326")

# Export as GeoJSON
gdf.to_file("tree_volumes.geojson", driver="GeoJSON")